In [1]:
## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
## for processing
import re
import nltk
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for w2v
import gensim
import gensim.downloader as gensim_api
## for bert
import transformers
import pandas as pd
import tarfile
import os
import gc
import random

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score


import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D,Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, Callback, ReduceLROnPlateau

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows',5000)
from tqdm.autonotebook import tqdm

import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [2]:
import pandas as pd 

data = pd.read_csv("/kaggle/input/reviews-red-sfr/RED_VF.csv", encoding='utf-8')
pd.set_option('display.max_colwidth', None)
data=data.drop(['Rating'] , axis=1)
data.head()

,Review
0,"Habitant à une trentaine de kilomètres de la frontière suisse j’ai eu le droit à un hors forfait de plus de 80€.\nEt Sfr ne m’a jamais prévenu que j’étais sur le réseau Suisse et pour causse je n’avais jamais eu de soucis précédemment !\nÇa fait mal de se faire voler comme cela, c’est de l’extorsion de fond!"
1,"Horrible.\n\n\n4g qui bug 24/24h, service client inexistant (ce sont des freelance qui ne parlent même pas bien le français et qui ne vous aideront à rien donc vous êtes seul derrière votre écran si vous avez un problème)\n\narnaque."
2,"J’ai souscris à une offre sport et résilier cette dernière avant la fin du premier mois d’inscription (moins de 30 jours).\nMais ils m’ont facturés quand même le deuxième mois sous prétexte que il faut résilier l’abonnement avant la fin du mois calendaire. Une fois basculé sur un nouveau mois calendaire, ils ont pour obligation de facturer même si on s’est désabonné avant que cela fasse 30 jours … C’est la première fois que je vois un tel fonctionnement. Mais c’est pas tout, j’ai même pas accès au service 😂\nDonc ils me prennent de l’argent mais j’ai pas le service. C’est pas beau ça ?\nMerci REDBYSFR."
3,"Minable! Et encore je suis gentil. Impossible de recevoir et d'envoyer un SMS. Impossible de joindre un conseillé car il faut passer par des informations en ligne, informations qui ne servent absolument à rien. N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux. Je viens de prendre un forfait chez RED by SFR il y a 8 jours... Je retourne chez mon ancien fournisseur. Au moins je vais avoir un forfait qui fonctionne et la possibilité d'avoir un vrai conseillé en ligne en cas de problème."
4,"Au mois de février, un démarcheur, M. Sadou MANE nous as contacté afin que l'on passe notre Box internet + forfaits mobile chez Red by SFR.\nIl nous a proposé une BOX internet et vendu 2 forfaits mobiles à 10€ chaque.\nSurprise, nous avons reçus en février et mars des factures de 16.99€ pour chaun de nos forfaits mobile au lieu de 10€ convenu par téléphone.\nJ'ai passé mon temps à contacter les services clients, fidélisation, résiliation verdict : on ne peux rien faire cette offre n'existe pas. C'est un scandale ! On nous vend un produit qui n'existe pas et on ne peux plus rien faire ? C'est une vraie arnaque. J'ai demandé à faire réécouter ma conversation téléphonique, mais personne ne prends en compte ma demande. Impossible de joindre un responsable. Un scandale !"


In [3]:
data.shape

(3452, 1)

In [4]:
import re
import string
import emoji
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text, stem=False):

    # Remove accents from text
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")

    # Remove emojis from text
#     text = emoji.get_emoji_regexp().sub(u'', text)

    # Remove URLs and email addresses from text
    text = re.sub(r'\b(?:https?://|www\.)\S+\b', '', text)
    text = re.sub(r'\b[\w.]+?@\w+?\.\w{2,4}\b', '', text)

    # Remove special characters from text
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Remove punctuation from text
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize text into words
    words = nltk.word_tokenize(text)

    # Remove stopwords from text
    stop_words = set(stopwords.words('french'))
    words = [word for word in words if word not in stop_words]

    # Apply stemming to text
    if stem:
        stemmer = FrenchStemmer()
        words = [stemmer.stem(word) for word in words]

    # Join words back into a string
    text = ' '.join(words)

    return text

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
data_cleaned = pd.DataFrame()
data_cleaned['Review'] = data['Review'].apply(preprocess_text)
data_cleaned.head()

,Review
0,Habitant a trentaine kilometres frontiere suisse jai droit a hors forfait plus 80 Et Sfr jamais prevenu jetais reseau Suisse causse navais jamais soucis precedemment Ca fait mal faire voler comme cela cest lextorsion fond
1,Horrible 4g bug 2424h service client inexistant freelance parlent meme bien francais aideront a rien donc etes seul derriere ecran si probleme arnaque
2,Jai souscris a offre sport resilier cette derniere avant fin premier mois dinscription moins 30 jours Mais mont factures quand meme deuxieme mois sous pretexte faut resilier labonnement avant fin mois calendaire Une fois bascule nouveau mois calendaire obligation facturer meme si sest desabonne avant cela fasse 30 jours Cest premiere fois vois tel fonctionnement Mais cest tout jai meme acces service Donc prennent largent jai service Cest beau ca Merci REDBYSFR
3,Minable Et encore gentil Impossible recevoir denvoyer SMS Impossible joindre conseille car faut passer informations ligne informations servent absolument a rien Nesperez trouver laide aupres conseillers SFR boutique car envoient roses car netes client chezeux Je viens prendre forfait chez RED by SFR a 8 jours Je retourne chez ancien fournisseur Au moins vais avoir forfait fonctionne possibilite davoir vrai conseille ligne cas probleme
4,Au mois fevrier demarcheur M Sadou MANE contacte afin lon passe Box internet forfaits mobile chez Red by SFR Il a propose BOX internet vendu 2 forfaits mobiles a 10 chaque Surprise recus fevrier mars factures 1699 chaun forfaits mobile lieu 10 convenu telephone Jai passe temps a contacter services clients fidelisation resiliation verdict peux rien faire cette offre nexiste Cest scandale On vend produit nexiste peux plus rien faire Cest vraie arnaque Jai demande a faire reecouter conversation telephonique personne prends compte demande Impossible joindre responsable Un scandale


In [6]:
pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d192b5228f5672b2b415af36d08b66627bf6d904591b945fe0374ffa7b902076
  Stored in directory: /root/.cache/pip/wheels/5b/e0/77/05245143a5b31f65af6a21f7afd3219e9fa4896f918af45677
Successfully built sacremoses
Note: you may need to restart the kernel to use updated packages.


In [7]:
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import TFAutoModel, AutoTokenizer
import transformers as trf


tokenizer = AutoTokenizer.from_pretrained("flaubert/flaubert_large_cased")
model = TFAutoModel.from_pretrained("flaubert/flaubert_large_cased",from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFFlaubertModel: ['pred_layer.proj.weight', 'pred_layer.proj.bias']
- This IS expected if you are initializing TFFlaubertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFFlaubertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFFlaubertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFFlaubertModel for predictions without further training.


In [8]:
def bert_embedding(txt):
    idx = tokenizer.encode(txt, max_length=1024,padding=True, truncation=True) # creating tokens
    if len(idx) >= 1024:
        return None
    idx = np.array(idx)[None,:] # converting to 2d array

    emb = model(idx) # bert layer
    hidden = np.array(emb[0][0]) # batch output of last_hidden_state

    sent_emb = hidden.mean(0) # creating mean vector
    return sent_emb

In [9]:
from tqdm.autonotebook import tqdm

sent_matrix = np.array([bert_embedding(text) for text in tqdm(data['Review'][0:1000])] , dtype=object)
sent_matrix.shape

  0%|          | 0/1000 [00:00<?, ?it/s]

(1000, 1024)

In [10]:
from tqdm.autonotebook import tqdm

sent_matrix1 = np.array([bert_embedding(text) for text in tqdm(data['Review'][1001:2000])] , dtype=object)
sent_matrix1.shape

  0%|          | 0/999 [00:00<?, ?it/s]

(999, 1024)

In [11]:
from tqdm.autonotebook import tqdm

sent_matrix2 = np.array([bert_embedding(text) for text in tqdm(data['Review'][2500:3000])] , dtype=object)
sent_matrix2.shape

  0%|          | 0/500 [00:00<?, ?it/s]

(500, 1024)

In [12]:
sent_matrix = np.concatenate((sent_matrix, sent_matrix1,sent_matrix2 ))

In [13]:
sent_matrix.shape

(2499, 1024)

In [14]:
import gensim

# Load the pre-trained FastText embeddings from the file
model1 = gensim.models.KeyedVectors.load_word2vec_format('/kaggle/input/fasttext-french-2b-300d/cc.fr.300.vec', binary=False)

# Find the most similar words to a given word
word = 'chat'
similar_words = model1.most_similar(word)

# Print the most similar words and their cosine similarity scores
print(f"Words most similar to '{word}':")
for similar_word, similarity_score in similar_words:
    print(f"{similar_word}: {similarity_score:.3f}")

Words most similar to 'chat':
chaton: 0.756
chats: 0.738
matou: 0.730
chien: 0.728
chat.: 0.705
Chat: 0.696
chat.Le: 0.681
chatUn: 0.668
chat-chien: 0.660
félin: 0.654


In [15]:
word = 'couverture'
similar_words = model1.most_similar(word , topn=10)

# Create a list of similar words
débit  = [similar_word for similar_word, similarity_score in similar_words]

# Print the list of similar words
print(f"Similar words to '{word}': {débit}")

Similar words to 'couverture': ['Couverture', 'couvertures', 'couverture.', 'couv', '.Couverture', 'sous-couverture', 'converture', 'couvertur', 'courverture', 'non-couverture']


In [16]:
RESIL = ["Très déçu de red SFR box à fuir.",
          "j'ai donc dû changer d'opérateur",
         "Je pars donc de chez eux car j'en peux plus, c'est le coup de grâce !! Se sont des voleurs !!! A fuir !!!",
         "Cette fois l'opérateur, qui appelle avec 45 min de retard, me propose d'annuler ma commande et d'en passer une autre (tout recommencer depuis le début).",
        "Quitté pour un autre opérateur avec une portabilité le 11 janvier.",
         "déjà j'avais quitté red mobile quand j'avais 50 go à 10 euro qui sont passés a plus de 13 euro, pour prendre red 5 go a 5 euro.",
          "Fuyez ou changez.",
          "A fuir, je ne recommande pas, on se sent volé!",
          "J’ai essayé plusieurs fois de résilier",
         "Je pense changer d’opérateur.",
         "Je me suis retrouvée avec mon contrat résilié et un nouveau contrat en place. J'ai découvert ça via mail, après l'appel.",
          "Lorsque je demande une résiliation sans frais ils me font passer de service en service et finissent par raccrocher après la musique d'attente (ça m'est arrivée deux fois).",
         "J'ai demandé la résiliation de mon contrat box Internet le 14 juin",
          "je suis dépité et bloqué dans une boucle RED by SFR",
         "faites demi-tour! Plus jamais je ferai confiance à ces incapables.",
         "Le client exprime son souhait de résilier son offre et mentionne que l'opérateur demande des frais de résiliation.",
         "Alors pour la résiliation bordel j’ai jamais vu ce niveau d’incompétence. Ils font exprès de ne pas vous fournir le bordereau de retour afin de vous facturer des frais supplémentaire.",
        "Changer de forfait dès que possible avant qui vous la mettre bien…",
         "J'ai demandé la résiliation de mon contrat box Internet le 14 juin.",

        ]

SERVICE_CLI = [ "Impossible de joindre un conseillé car il faut passer par des informations en ligne, informations qui ne servent absolument à rien. N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux."
,"Le client a contacté plusieurs services clients, fidélisation et résiliation pour régler ce problème, mais n'a obtenu aucune réponse satisfaisante. Le client exprime sa frustration face à l'impossibilité de joindre un responsable ou de faire réécouter sa conversation téléphonique."
,"Le client estime avoir été trompé sur l'offre proposée par le démarcheur de Red by SFR et considère cela comme une arnaque. Le client a été vendu un produit qui n'existe pas et ne peut rien faire pour résoudre cette situation."              
,"Le 20/03/23 plus de communication ni sortante ni entrante. SFR a coupé le ligne arbitrairement sans prévenir pour motif de problème lié a la sureté et le réclame des documents déjà fourni(RIB, Justificatif domicile, mail de confirmation sfr ouverture de ligne et pièce identité). Ce qui est inadmissible: couper une ligne sans prévenir ni demander des éventuels documents manquants."              
,"Contact par chat uniquement avec des conseillers incompétents qui ne savent pas pourquoi la demande est rejetée, et qui vous renvoient au site ou à faire un courrier à SFR. Le service client RED s'est fortement inspiré du sketch Sens interdit de Raymond Devos, pour traiter les demandes.",
"Service client deplorable. Après plusieurs minutes à essayer d'avoir un details de facture et un suite à un changement de forfait la personne n'a jamais su expliqué correctement si ce n'est que dire que la facture est correcte.. Elle a fini par mettre fin à la conversation sans attendre réponse de ma part.",
"ont peut que communiquer avec eux que par message SMS",
"Vous recevrez les 15€ de réduction dès que le statut de votre commande d'abonnement aura été validé par le service RedBySfr.",
"Lorsque je les ai contacté … selon eux rien d'anormal donc ils ne remboursent pas !",
"Quand par miracle on arrive à rentrer en contact avec un technicien, il s'agit d'un call center à l'étranger, qui n'arrive à rien résoudre. Au dernier appel, mon interlocuteur m'a raccroché au nez. Lamentable.",
 "J'appelle ou je tchat quotidiennement avec le service clientèle qui n'est toujours pas en mesure de me donner un rdv pour installer et raccorder ma ligne.",
"Impossible de parler à un humain de vive voix ou de face à face.",
"Et impossible après 29 tentatives de les joindre pour remboursement de la période du 11 au 30. Drôle de manières !!!",
"c'est inacceptable", "je choisis les offres qui sont respectées a vie", "et red ne fait plus partie de ces revendeurs.",
"j'ai fait 6 demandes et de multiples appels et chat en ligne et la demande n'aboutit pas. Le SAV est totalement déficient !!!",
"Service client incompétent !!! [...] Quand nous avons eu le service client la personne s'en fou totalement elle ne veut rien savoir elle nous demande de nous débrouiller !! Vraiment n'y aller pas ces scandaleux.",
"un coup de fil, et généralement le problème est résolu rapidement",
"Service client d’une incompétence… à ce niveau là c’est du jamais vu. Une fois sur 2 on vous raccroche au nez. On fait des erreurs sur votre contrat… incroyable! Fuyez ou changez.",
"J'ai passé des heures et des heures à me faire balader de service en service, à répéter toujours la même histoire et mes coordonnées, pour que les gens ne comprennent rien, on ne les entend pas, ça raccroche! Tout ce que vous ne voulez pas vivre avec un call center.",
"Service client incompetent",
"Au bout de plus d’un mois à joindre leurs services quotidiennement.",
"J'ai contacté SFR pour un problème de ma box.",
"Le service consommateur lit un script et ne sait pas quoi dire si on sort du script.",
"Aucun vrai conseiller, seulement un Bot. ça fait deux heures que j'essaie de résoudre un problème, et toutes les deux minutes ça se réinitialise et me repose les mêmes questions.",
"SAV compétent et professionnel",
"30 minutes d'attente" ,
"pas de disponibilité pour un dépannage de box",
"des dizaines d'échanges avec le service client, sur l'application et par téléphone",



]



PROMO = [ "Pub TV et sans prévenir le client et sans laisser le choix au client",
"Je vous propose de profitez de 15€ offerts lors de votre souscription d'abonnement chez Red. (15€ pour Red Box et 10€ pour Red mobile)",
"quand on sait que a la poste le forfait est a 9.99 euro pour 50 go dont 15 go internationale et A VIE.",
 "Et lorsque vous avez quelqu'un, il vous promet un geste commercial que vous n'avez jamais, simplement pour se débarrasser de vous!",
"là je viens d'avoir un appel pour me soit disant m'offrir un bouquet famille avec Netflix mais au final ils ne m'offrent ni Netflix ni le bouquet, super de faire perdre du temps aux gens pour rien.",
"Pour mes deux offres, je me retrouve régulièrement avec des options pour lesquelles je n'ai fais aucunes demandes",
"Il vous change votre abonnement sans vous prévenir pour une meilleure offre sois disant avec une prime en plus de 3 €",
"Je prends un abonnement + achat d'un téléphone pour SFR RED 200 GO; la ligne est opérationnelle le 18/03/23 après réception du matériel et carte Sim. Tout fonctionne bien."

]

FACTURATION = ["Les factures reçues sont supérieures à ce qui a été convenu par téléphone avec le démarcheur de Red by SFR. Le client est mécontent de cette situation et considère cela comme une arnaque.",
"Après plusieurs minutes à essayer d'avoir un details de facture et un suite à un changement de forfait la personne n'a jamais su expliqué correctement si ce n'est que dire que la facture est correcte.",
"mes 3 forfaits 200Go sont passés à 28 euros",
"Le nombre de fois que je me suis retrouvée avec des factures augmentées on ne sait pas pourquoi... Ils vous rajoutent des options sans que vous ayez rien demandé !",
"Ils m'ont changé mon forfait sans aucune demande de ma part (toujours pareil sans aucun consentement de ma part), bien évidemment beaucoup plus cher !!!!",
"Quitté pour un autre opérateur avec une portabilité le 11 janvier, ils retirent sur ton compte l'intégralité forfait de janvier et des portions telles que Deezer.",
"j'ai la box red sfr adsl 15 euro a vie qui est passé a 16 euro et mon mobile 5 go 5euro qui est passé a 6 euro ce mois ci",
 "ont tendance à oublier des options gratuite lors de déménagement de lignes",
"Leur prix augmentent sans notification de leur part de quelques euros chaque année.",
"tous les mois paye et non remboursé pour une box dans un carton.",
"surfacturation mensuelle d'une option tv que je n ai pas",
"Prix en perpetuelle augmentation",
"Elle n'a pas été effective et j'ai été facturée les mois de juin, juillet et août. À ce jour, donc en octobre, j'attends toujours le remboursement après des dizaines d'échanges avec le service client, sur l'application et par téléphone, qui me confirment à chaque fois que je vais être remboursée mais ce est absolument pas le cas. attends toujours.",
"J'ai été facturée les mois de juin, juillet et août.",
"Le client est facturé malgré l'absence de service Internet.",
"Je me retrouve donc certains mois avec +3€ par-ci, +5€ par là.",

    





]

SPEED_NET = [ "Minable! Et encore je suis gentil. Impossible de recevoir et d'envoyer un SMS."
,"pas moyen de passer un coup de fil sans coupure, ils m'ont dit configurer en appels wifi pour éviter le probleme, ok mais dans ce cas à quoi cela sert il que je paye un abonnement chez eux. minable"
,"Notre entreprise travaille sans internet alors que c'est le seul outil nécessaire.",
 "ces 4g box sont les plus éclatées que j'ai jamais eu de ma vie",
 "même sans ces bugs la connexion n'est pas stable",
"1 semaine de télétravail et je me retrouve avec deux incidents, partage de connection, utilisation de mon forfait internet mobile pour compenser leur manque de réactivité.",




]

TECH_INSTALL = ["Problèmes de débit et coupures intempestives du réseau internet depuis l'installation de la box (4 mois). Chatbot pour résoudre le problème qui tourne en rond.",
                "Le 08/03, on m'appelle et on me dit que c'est ma ligne personnelle qui a un pb, un technicien est programmé pour venir le lendemain 09/03 entre 10h et 12h. Évidemment personne ne vient....",
                "J'appelle ou je tchat quotidiennement avec le service clientèle qui n'est toujours pas en mesure de me donner un rdv pour installer et raccorder ma ligne.",
                 "Problème sur ma fibre optique, 2 semaines d'attentes pour avoir un technicien.",
                "Jour de congé posé pour être présente, aucune trace du technicien le jour J.",
                "Jour de congé posé pour rien, technicien absent, problème toujours présent.",
                "Et les techniciens sont tous en sous-traitances ce qui fait qu'ils ne sont pas investis.",
                 "Le technicien ne vient pas au rdv.",
                 "Des rendez-vous non honorés de la part des techniciens, un problème qui vient toujours d’ailleurs.",
                "Problèmes techniques récurrents avec la fibre. Ça fait 3 mois que nous l'utilisons, nous en sommes à notre 5ème dépannage pour coupure l'Internet (dont 2 la même semaine).",
                "30 minutes d'attente technicien qui ne se présente pas et pas de disponibilité pour un dépannage de box qui coupe sans arrêt",
                "technicien qui ne se présente pas",
                "box qui coupe sans arrêt",
                "il faut appeler un technicien, qui la plupart du temps met plus d'une semaine à arriver et ne sert à rien",
                
                
                
    ]


CONTRAT = [
"Une grosse galère pour faire un simple changement de titulaire sur une ligne existante, via le site RED. Rejet 3 fois de suite au bout de 7j sans explication plausible."   ,
"Tiens tiens tiens ... 2 mois.... c'est justement le délai légal dans lequel le FAI doit impérativement avoir effectué le raccordement. Au delà il est susceptible d'avoir des problèmes juridiques et comme par hasard on me propose ce jour là d'annuler et recommencer la procédure ...",
 "une règle de 3 fait que mon abonnement 5 go dont 2 go internationale devrait être a 1 euro au lieu de 5 euro et il se permette de rajouter 1 euro d'augmentation par mois.",
 "ont tendance a vouloir imposer des options",
 "On fait des erreurs sur votre contrat.",
"Chez Red by SFR depuis quelques années, force est de constater que la fidélité ne paye pas même pour un service sans engagement.",
"lors de ma souscription j'étais sous liste rouge,",
"Je sais pertinemment que l'un n'a pas à voir avec l'autre puisque j'étais auparavant chez free et on m'avait une fois changé la box sans pour autant changer le contrat.",
"rien de cela n'est stipulé dans mon contrat",
"J'ai demandé la résiliation de mon contrat box Internet le 14 juin.",


]
    
FIBRE = ["Problème sur ma ligne fixe FIBRE apparu le 1 octobre",
        "Problème sur ma fibre optique",
        "la fibre plante un jour sur deux.",
        " Le client a une offre fibre",
        "pres 6 mois de fibre chez Red by Sfr"]

NETWORK = [". Le 20/03/23 plus de communication ni sortante ni entrante. SFR a coupé le ligne arbitrairement sans prévenir pour motif de problème lié a la sureté et le réclame des documents déjà fourni(RIB, Justificatif domicile, mail de confirmation sfr ouverture de ligne et pièce identité). Ce qui est inadmissible: couper une ligne sans prévenir ni demander des éventuels documents manquants. Cette coupure de ligne met en danger des personnes vivants ds une région isolée. C est un procédé de voyous.",
           "Un des pires opérateurs que j'ai connu (1 an) en terme de réseau, d'incidents techniques.",
            "Le client rencontre des problèmes de coupure de fibre.",
           "pour une couverture de 5g 😂😂 ou la ville et même pas concerner"
          ]


In [17]:
RESIL = [preprocess_text(text) for text in RESIL]
SERVICE_CLI = [preprocess_text(text) for text in SERVICE_CLI]
PROMO = [preprocess_text(text) for text in PROMO]
FACTURATION = [preprocess_text(text) for text in FACTURATION]
SPEED_NET = [preprocess_text(text) for text in SPEED_NET]
TECH_INSTALL = [preprocess_text(text) for text in TECH_INSTALL]
CONTRAT = [preprocess_text(text) for text in CONTRAT]
FIBRE = [preprocess_text(text) for text in FIBRE]
NETWORK = [preprocess_text(text) for text in NETWORK]

In [18]:
FACTURATION = np.array([bert_embedding(text) for text in FACTURATION])
SERVICE_CLI = np.array([bert_embedding(text) for text in SERVICE_CLI])
TECH_INSTALL = np.array([bert_embedding(text) for text in TECH_INSTALL])
SPEED_NET = np.array([bert_embedding(text) for text in SPEED_NET])
RESIL = np.array([bert_embedding(text) for text in RESIL])
PROMO = np.array([bert_embedding(text) for text in PROMO])
CONTRAT = np.array([bert_embedding(text) for text in CONTRAT])
FIBRE = np.array([bert_embedding(text) for text in FIBRE])
NETWORK = np.array([bert_embedding(text) for text in NETWORK])

In [19]:
NETWORK.shape

(4, 1024)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

for d in [FACTURATION, SERVICE_CLI, TECH_INSTALL, SPEED_NET, RESIL, PROMO, CONTRAT, FIBRE, NETWORK]:
    d_emb = [t for t in d]
    print('Cosine similarity of given samples: {}'.format(cosine_similarity(d_emb[1][None, :], d_emb[2][None, :])))

Cosine similarity of given samples: [[0.8377405]]
Cosine similarity of given samples: [[0.9468978]]
Cosine similarity of given samples: [[0.9173139]]
Cosine similarity of given samples: [[0.81398284]]
Cosine similarity of given samples: [[0.7877507]]
Cosine similarity of given samples: [[0.82409596]]
Cosine similarity of given samples: [[0.7372923]]
Cosine similarity of given samples: [[0.6534023]]
Cosine similarity of given samples: [[0.80332077]]


In [29]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_score = pd.DataFrame()
cosine_score['id'] = range(len(sent_matrix))
cosine_score['Facturation'] = cosine_similarity(sent_matrix, FACTURATION.mean(0)[None, :])
cosine_score['Service client'] = cosine_similarity(sent_matrix, SERVICE_CLI.mean(0)[None, :])
cosine_score['Installation technique'] = cosine_similarity(sent_matrix, TECH_INSTALL.mean(0)[None, :])
cosine_score['Vitesse d\'Internet'] = cosine_similarity(sent_matrix, SPEED_NET.mean(0)[None, :])
cosine_score['Contrat'] = cosine_similarity(sent_matrix, CONTRAT.mean(0)[None, :])
cosine_score['Fibre'] = cosine_similarity(sent_matrix, FIBRE.mean(0)[None, :])
cosine_score['Couverture réseau'] = cosine_similarity(sent_matrix, NETWORK.mean(0)[None, :])
cosine_score['Résilience'] = cosine_similarity(sent_matrix, RESIL.mean(0)[None, :])
cosine_score['Promotion'] = cosine_similarity(sent_matrix, PROMO.mean(0)[None, :])

# #Obtenir les catégories avec un score de similarité cosinus supérieur à 0.7 pour chaque phrase
# cosine_score['top_categories'] = cosine_score[['Facturation', 'Service client', 'Installation technique', 'Vitesse d\'Internet', 'Contrat', 'Fibre', 'Couverture réseau', 'Résilience', 'Promotion']].apply(lambda x: [label for label in x.index if x[label] > 0.7], axis=1)

# # Si aucune catégorie n'a un score de similarité cosinus supérieur à 0.7, ajouter "None" dans la colonne "top_categories"
# cosine_score['top_categories'] = cosine_score['top_categories'].apply(lambda x: x if len(x) > 0 else ['None'])

cosine_score['top_categories'] = cosine_score[['Facturation', 'Service client', 'Installation technique', 'Vitesse d\'Internet','Contrat','Fibre','Couverture réseau','Résilience', 'Promotion']].apply(lambda x: x.nlargest(3).index.tolist(), axis=1)


cosine_score.head()

,id,Facturation,Service client,Installation technique,Vitesse d'Internet,Contrat,Fibre,Couverture réseau,Résilience,Promotion,top_categories
0,0,0.653859,0.732341,0.666603,0.657801,0.730903,0.604712,0.650117,0.666851,0.714528,"[Service client, Contrat, Promotion]"
1,1,0.660188,0.747929,0.687045,0.675716,0.731800,0.611975,0.660732,0.672633,0.722867,"[Service client, Contrat, Promotion]"
2,2,0.737119,0.756893,0.732885,0.737262,0.753508,0.697694,0.710752,0.737391,0.746449,"[Service client, Contrat, Promotion]"
3,3,0.742563,0.770562,0.742578,0.753309,0.764377,0.704731,0.722786,0.752540,0.759361,"[Service client, Contrat, Promotion]"
4,4,0.759743,0.790503,0.755983,0.761432,0.782409,0.715623,0.738022,0.763133,0.776290,"[Service client, Contrat, Promotion]"


In [30]:
import gc

# cosine_score['top_categories'] = cosine_score[['Facturation/resiliance','Service Client','Technique/Installation',"Changement d'offre"]].idxmax(axis=1) #finding the column which has maximum value and retunrning the column name (this becomes the label for the text)
df = cosine_score

del cosine_score #deleting as we dont need this df anymore
gc.collect()

df.head(30)

,id,Facturation,Service client,Installation technique,Vitesse d'Internet,Contrat,Fibre,Couverture réseau,Résilience,Promotion,top_categories
0,0,0.653859,0.732341,0.666603,0.657801,0.730903,0.604712,0.650117,0.666851,0.714528,"[Service client, Contrat, Promotion]"
1,1,0.660188,0.747929,0.687045,0.675716,0.731800,0.611975,0.660732,0.672633,0.722867,"[Service client, Contrat, Promotion]"
2,2,0.737119,0.756893,0.732885,0.737262,0.753508,0.697694,0.710752,0.737391,0.746449,"[Service client, Contrat, Promotion]"
3,3,0.742563,0.770562,0.742578,0.753309,0.764377,0.704731,0.722786,0.752540,0.759361,"[Service client, Contrat, Promotion]"
4,4,0.759743,0.790503,0.755983,0.761432,0.782409,0.715623,0.738022,0.763133,0.776290,"[Service client, Contrat, Promotion]"
5,5,0.779701,0.825593,0.791679,0.795076,0.816672,0.745649,0.780134,0.797014,0.818150,"[Service client, Promotion, Contrat]"
6,6,0.755334,0.777890,0.755479,0.767102,0.764081,0.715389,0.739934,0.758855,0.756811,"[Service client, Vitesse d'Internet, Contrat]"
7,7,0.726109,0.761650,0.736086,0.733496,0.757024,0.681307,0.703178,0.740371,0.746377,"[Service client, Contrat, Promotion]"
8,8,0.695952,0.763275,0.706967,0.709150,0.753391,0.644257,0.693465,0.699707,0.738817,"[Service client, Contrat, Promotion]"
9,9,0.722392,0.762933,0.726406,0.728319,0.755654,0.679271,0.702701,0.730441,0.748228,"[Service client, Contrat, Promotion]"


In [31]:
df['Review'] = data['Review'][0:1000] # Earlier we have defined num_sentences
df.drop(['Facturation', 'Service client', 'Installation technique', 'Vitesse d\'Internet', 'Résilience', 'Promotion'], axis=1, inplace=True)
df1 = df[['id', 'Review', 'top_categories']]
df1.head(50)

,id,Review,top_categories
0,0,"Habitant à une trentaine de kilomètres de la frontière suisse j’ai eu le droit à un hors forfait de plus de 80€.\nEt Sfr ne m’a jamais prévenu que j’étais sur le réseau Suisse et pour causse je n’avais jamais eu de soucis précédemment !\nÇa fait mal de se faire voler comme cela, c’est de l’extorsion de fond!","[Service client, Contrat, Promotion]"
1,1,"Horrible.\n\n\n4g qui bug 24/24h, service client inexistant (ce sont des freelance qui ne parlent même pas bien le français et qui ne vous aideront à rien donc vous êtes seul derrière votre écran si vous avez un problème)\n\narnaque.","[Service client, Contrat, Promotion]"
2,2,"J’ai souscris à une offre sport et résilier cette dernière avant la fin du premier mois d’inscription (moins de 30 jours).\nMais ils m’ont facturés quand même le deuxième mois sous prétexte que il faut résilier l’abonnement avant la fin du mois calendaire. Une fois basculé sur un nouveau mois calendaire, ils ont pour obligation de facturer même si on s’est désabonné avant que cela fasse 30 jours … C’est la première fois que je vois un tel fonctionnement. Mais c’est pas tout, j’ai même pas accès au service 😂\nDonc ils me prennent de l’argent mais j’ai pas le service. C’est pas beau ça ?\nMerci REDBYSFR.","[Service client, Contrat, Promotion]"
3,3,"Minable! Et encore je suis gentil. Impossible de recevoir et d'envoyer un SMS. Impossible de joindre un conseillé car il faut passer par des informations en ligne, informations qui ne servent absolument à rien. N'espérez pas trouver de l'aide auprès des conseillers SFR dans une boutique car ils vous envoient sur les roses car vous n'êtes pas client chez-eux. Je viens de prendre un forfait chez RED by SFR il y a 8 jours... Je retourne chez mon ancien fournisseur. Au moins je vais avoir un forfait qui fonctionne et la possibilité d'avoir un vrai conseillé en ligne en cas de problème.","[Service client, Contrat, Promotion]"
4,4,"Au mois de février, un démarcheur, M. Sadou MANE nous as contacté afin que l'on passe notre Box internet + forfaits mobile chez Red by SFR.\nIl nous a proposé une BOX internet et vendu 2 forfaits mobiles à 10€ chaque.\nSurprise, nous avons reçus en février et mars des factures de 16.99€ pour chaun de nos forfaits mobile au lieu de 10€ convenu par téléphone.\nJ'ai passé mon temps à contacter les services clients, fidélisation, résiliation verdict : on ne peux rien faire cette offre n'existe pas. C'est un scandale ! On nous vend un produit qui n'existe pas et on ne peux plus rien faire ? C'est une vraie arnaque. J'ai demandé à faire réécouter ma conversation téléphonique, mais personne ne prends en compte ma demande. Impossible de joindre un responsable. Un scandale !","[Service client, Contrat, Promotion]"
5,5,"Je prends un abonnement + achat d'un téléphone pour SFR RED 200 GO; la ligne est opérationnelle le 18/03/23 après réception du matériel et carte Sim. Tout fonctionne bien. Le 20/03/23 plus de communication ni sortante ni entrante. SFR a coupé le ligne arbitrairement sans prévenir pour motif de problème lié a la sureté et le réclame des documents déjà fourni(RIB, Justificatif domicile, mail de confirmation sfr ouverture de ligne et pièce identité). Ce qui est inadmissible: couper une ligne sans prévenir ni demander des éventuels documents manquants. Cette coupure de ligne met en danger des personnes vivants ds une région isolée. C est un procédé de voyous.","[Service client, Promotion, Contrat]"
6,6,"RED by SFR.\n\nJe suis client de cette société SFR depuis longtemps : RED by SFR, mais depuis quelques mois, rien ne va plus, le service client est bien en dessous de ce qu’il devrait être. Le dernier appel que j’ai fait s’est mal termine et ma demande n’a pas été réglée. On a au bout du fil des personnes bien souvent incompétentes ou qu’ils n’écoutent pas ce qu’on leur dit. Mes problèmes n’ont jamais été réglés, et j’ai même fortement pensé à changer de fournisseur. De plus, depuis quelques mois, la rapidité 

In [32]:
df1.to_csv('output1.csv')